In [14]:
import pandas as pd
from pydantic import BaseModel

In [15]:
from glob import glob

folders = glob("./features/*")
folders.sort()
folders

print(folders)

['./features/p0-9999', './features/p10000-19999', './features/p20000-29999', './features/p30000-39999', './features/p50000-59999', './features/p60000-69999', './features/p70000-79999']


In [16]:
df = pd.read_parquet(f"./features/p0-9999/encoded_data.parquet")

In [18]:
df['title']

0        Quyết định 2054/QĐ-UBND năm 2024 về Khung kế h...
1        Quyết định 1396/QĐ-UBND năm 2024 về Khung kế h...
2        Quyết định 1708/QĐ-UBND năm 2024 về Khung kế h...
3        Quyết định 590/QĐ-QLD năm 2024 về Danh mục 08 ...
4        Thông tư 02/2024/TT-VKSNDTC quy định về công t...
                               ...                        
10007    Quyết định 1015/QĐ-UBND phê duyệt kế hoạch sử ...
10008    Quyết định 1433/QĐ-UBND năm 2024 quy định về v...
10009    Chỉ thị 04/CT-UBND năm 2024 tăng cường công tá...
10010    Kế hoạch 83/KH-UBND tuyên truyền về quản lý tà...
10011    Quyết định 817/QĐ-BGDĐT năm 2024 công bố công ...
Name: title, Length: 10012, dtype: object

In [1]:
from pymilvus import (
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
    connections,
    AnnSearchRequest,
    RRFRanker,
    MilvusClient
)

In [2]:
from pymilvus.model.hybrid import BGEM3EmbeddingFunction
import torch

device= 'cuda' if torch.cuda.is_available() else 'cpu'

embedding = BGEM3EmbeddingFunction(
    'BAAI/bge-m3',  # Specify the model name
    device=device,  # Specify the device to use, e.g., 'cpu' or 'cuda:0'
    # Specify whether to use fp16. Set to `False` if `device` is `cpu`.
    use_fp16=False,
)
def get_embeddings(text):
    res = embedding.encode_queries(text)
    return res

/home/datltt/miniconda3/envs/rag_prj_2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 177224.11it/s]
/home/datltt/miniconda3/envs/rag_prj_2/lib/python3.10/site-packages/FlagEmbedding/BGE_M3/modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be l

In [5]:
res = get_embeddings(["Hello world"])

In [8]:
print("Sparse query dim:", embedding.dim["sparse"], list(res["sparse"])[0].shape)

Sparse query dim: 250002 (1, 250002)


In [10]:
connections.connect(host = "localhost",
                   port = 19530,
                   db_name = "rag_search",
                  )
collection = Collection("thu_vien_phap_luat")

In [52]:
rerank = RRFRanker()

def search(collection, text, limit=3):
    query_embeds = embedding.encode_queries([text])

    title_sparse_search_params = {
        "data": query_embeds["sparse"],  # Query vector
        "anns_field": "title_sparse",  # Vector field name
        "param": {
            "metric_type": "IP",  # This parameter value must be identical to the one used in the collection schema
        },
        "limit": limit,  # Number of search results to return in this AnnSearchRequest
    }
    title_sparse_request = AnnSearchRequest(**title_sparse_search_params)

    title_dense_search_params = {
        "data": query_embeds["dense"],
        "anns_field": "title_dense",
        "param": {
            "metric_type": "COSINE",
            "ef": 20,
        },
        "limit": limit,
    }
    title_dense_request = AnnSearchRequest(**title_dense_search_params)

    content_sparse_search_params = {
        "data": query_embeds["sparse"],
        "anns_field": "content_sparse",
        "param": {
            "metric_type": "IP",
        },
        "limit": limit,
    }
    content_sparse_request = AnnSearchRequest(**content_sparse_search_params)

    content_dense_search_params = {
        "data": query_embeds["dense"],
        "anns_field": "content_dense",
        "param": {
            "metric_type": "COSINE",
            "ef": 20,
        },
        "limit": limit,
    }
    content_dense_request = AnnSearchRequest(**content_dense_search_params)

    reqs = [
        title_sparse_request,
        title_dense_request,
        content_sparse_request,
        content_dense_request,
    ]

    collection.load()
    res = collection.hybrid_search(
        reqs,  # List of AnnSearchRequests created in step 1
        rerank,  # Reranking strategy specified in step 2
        limit=limit,  # Number of final search results to return
        output_fields=["id", "title_text", "content_text", "url"],
    )[0]
    return res

In [97]:
from pydantic import BaseModel
class RetrievedDocument(BaseModel):
    url: str = ""
    title: str = ""
    content: str = ""

In [67]:
def build_chat_message(
        user_msg: str,
        system_msg: str | None = None,
        previous_conversation: str | None = None,
) -> str:
    messages: list[str] = []
    if system_msg:
        messages.append({"role": "system", "content": system_msg})
    messages.append({"role": "user", "content": user_msg})

    text: str = ""
    for turn in messages:
        turn["content"] = turn["content"].strip()
        text += "<|im_start|>{role}\n{content}<|im_end|>\n".format(**turn)

    text += "<|im_start|>assistant\n"

    if previous_conversation:
        if not previous_conversation.endswith("\n"):
            previous_conversation += "\n"
        text = previous_conversation + text
    return text


In [70]:
from langchain_community.llms import Ollama

llm_model =  Ollama(model="qwen2.5:latest")

In [101]:
import ollama
def llm_completion(system_prompt, user_query):
    messages = [
        {
            "role" : "System",
            "content": system_prompt
        },
        {
            "role" : "User",
            "content": user_query
        }
    ]
    response = ollama.chat(model='qwen2.5', messages=messages)
    return response['message']['content']

In [66]:
CLASSIFY_SYSTEM_PROMPT_TEMPLATE = \
    """You are a virtual assistant for {{classify_topic}}
        ## Rules
        1. Only answer `Yes` or `No`.
        2. Respond `Yes` if the question is related to {{classify_topic}} otherwise respond `No`.
        3. Reject description: Topics on criminal, civil, administrative documents, aviation, maritime, medical, and political issues not related to {{classify_topic}} should be answered with `No`.
    """

In [95]:
QUESTION_PARSING_SYS_MSG_PROMPT_LINE = """You are a legal assistant. Your task is to analyze questions from the Human.
Instructions:
1. You need to analyze the question below and break it down into 4 most correct smallest questions.
2. Each small question is displayed in a line folowing the format:
- <question1>
- <question2>
..."""

In [88]:
def parse_search_query(prompt):
    res = []
    for line in prompt.split("\n"):
        res.append(line.replace("-", "").strip())
            
    return res

In [108]:
query = "thông tin về quản lý, kết nối và chia sẻ dữ liệu số của cơ quan nhà nước"

In [131]:
## Search query 

context_items = [ RetrievedDocument(url = item.url, title = item.title_text, content = item.content_text) for item in search(collection, query, limit = 3)]

# reference = [
#     {"url": context_item['entity']['url'],
#      'title': context_item['entity']['title']}
#     for context_item in context_items]
context = [f"""post {index}:
           - context {item.content}
           - url: {item.url}
           ---
           """ for index, item in enumerate(context_items)]

prompt_formatted = '''
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question or history of the chat. If you don't know the answer, just say that you don't know. If you know return both answer and reference document (title and document url link) in user language. Use three sentences maximum and keep the answer concise.
    Question: {question}
    Context: {context}
    History:{history}
    Answer:

    '''.format(
    question=query, context=context, history="")

llm_res = llm_model.invoke(prompt_formatted)
print(llm_res)

E0000 00:00:1727865258.584028 1426062 ssl_transport_security.cc:1654] Handshake failed with fatal error SSL_ERROR_SSL: error:100000f7:SSL routines:OPENSSL_internal:WRONG_VERSION_NUMBER


Đây là một quyết định quan trọng về quy chế khai thác và sử dụng dữ liệu của tỉnh Hà Giang. Dưới đây là tóm tắt các điểm chính trong văn bản:

1. **Mục đích**: Đảm bảo việc khai thác, sử dụng, chia sẻ thông tin, dữ liệu được tuân thủ pháp luật.

2. **Nguyên tắc và trách nhiệm**:
   - Các cơ quan Nhà nước phải duy trì định kỳ hàng năm theo quy định của pháp luật.
   - Cơ sở dữ liệu do các cơ quan Nhà nước cung cấp trên Cổng thông tin dữ liệu dùng chung tỉnh Hà Giang phải đảm bảo quy định về bảo mật thông tin và không xâm phạm lợi ích cá nhân, tổ chức, Nhà nước.

3. **Quyền truy cập**:
   - Các cơ quan phối hợp được cấp phát tài khoản truy cập và phân quyền để cập nhật dữ liệu theo chức năng nhiệm vụ của mình.

4. **Yêu cầu khai thác, sử dụng dữ liệu**:
   - Tuân thủ quy định về bảo vệ thông tin cá nhân, bí mật Nhà nước, an toàn thông tin mạng.
   - Cơ quan Nhà nước phải chia sẻ dữ liệu cho cơ quan khác nếu có yêu cầu theo pháp luật. Không cung cấp thông tin qua hình thức văn bản đối với

In [109]:
parse_question = llm_completion(system_prompt=QUESTION_PARSING_SYS_MSG_PROMPT_LINE, user_query=query)
search_query = parse_search_query(parse_question)
print(parse_question)

- Thông tin về cách thức quản lý dữ liệu số của cơ quan nhà nước?
- Phương pháp kết nối dữ liệu số giữa các cơ quan nhà nước là gì?
- Cơ chế chia sẻ dữ liệu số giữa các cơ quan nhà nước ra sao?
- Pháp luật có quy định gì về việc quản lý, kết nối và chia sẻ dữ liệu số của cơ quan nhà nước?


In [102]:
USER_QAC_MESSAGE_TEMPLATE = """'''Cho đoạn văn:
{d}.
Câu hỏi: \"{q}\"

Đoạn văn trên có chứa câu trả lời cho câu hỏi không?'''"""


QAC_SYS_TEMPLATE = """Bạn được cung cấp một đoạn văn và một câu hỏi, hãy xác định xem đoạn văn có chứa câu trả lời cho câu hỏi không. Response {"is_answer": "yes"} nếu đoạn văn chứa câu trả lời cho câu hỏi.Response {"is_answer": "no"} nếu đoạn văn không chứa câu trả lời cho câu hỏi."""  # noqa: E501


In [114]:
def parse_qac_response(response):
    yes_sub = """yes"""""
    no_sub = """no"""""
    if yes_sub in response:
        return "yes"
    if no_sub in response:
        return "no"
    return "not found"

In [120]:
## Search query 
contexts = []
for query in search_query:
    retrieved_documents = [ RetrievedDocument(url = item.url, title = item.title_text, content = item.content_text) for item in search(collection, query, limit = 3)]
    for doc in retrieved_documents:
        user_query = USER_QAC_MESSAGE_TEMPLATE.format(q=query.strip(), d=doc.content.strip())
        ### QAC the search response
        qac_res = llm_completion(system_prompt=QAC_SYS_TEMPLATE, user_query=user_query)
        if parse_qac_response(qac_res) == "yes":
            contexts.append(doc)
        else:
            print(parse_qac_response(qac_res))

no
no
no


In [121]:
len(contexts)

9

In [123]:
thoughts = None

In [127]:
USER_MSG_TEMPLATE = """Câu hỏi: {q}
Search result: ```
{snippet}
```"""

FINAL_SYS_MSG = (
    "<role>\nYou are a legal question answering assistant. Your role is to provide accurate,"
    " relevant and well-structured responses to user queries by leveraging information"
    " from search results.\n\nTo generate a high-quality response:\n\n1. Carefully"
    " analyze the user's question to understand their information needs\n2. Review the"
    " search results returned by the system to identify relevant information \n3."
    " Synthesize the key details from the search results into a coherent, informative"
    " response that directly addresses the user's question\n4. Structure the response"
    " as follows:\n   - Opening (1-3 sentences): Provide a general overview of the"
    " topic without including specific numerical details \n   - Answer (detailed"
    " response): Present the main findings and details from the search results, using"
    " bullet points (-) to clearly list key information and penalties\n   - Citation:"
    " Include references to the search results at the end of the response\n5. Aim for a"
    " response length of 500-1000 words\n6. Write the response in Vietnamese  \n7."
    " Focus only on information directly relevant to answering the question\n8. Avoid"
    ' using generic phrases like "Based on the information found in the related'
    ' documents," "Your question is unclear," or "based on the information from'
    ' [QA-2]"\n\n<response_format>\n # Mở đầu\n...\n\n# Trả lời\n\n...\n</response_format>\n</role>'
)

In [128]:
def format_snippets(list_docs: list[RetrievedDocument]):
    text = ""
    for doc in list_docs:
        text += "\n---\n"
        text += doc.content.strip() + "\n\n"
        
    return text

In [129]:
snippet_text: str = format_snippets(
    contexts
)

user_msg: str = USER_MSG_TEMPLATE.format(
    q=query,
    snippet=" ".join(snippet_text.split(" ")[:5000]),
)

if thoughts is not None:
    user_msg += "\n---\nFollow these notes to answer:\n" + "\n".join(thoughts)
    
final_response = llm_completion(system_prompt=FINAL_SYS_MSG, user_query=user_msg)

print(final_response)

Dựa trên nội dung bạn cung cấp, tôi sẽ giúp tóm tắt và tổ chức lại các mục chính trong phần phát triển hạ tầng dữ liệu của tỉnh Trà Vinh. Dưới đây là bản đồ kế hoạch chi tiết:

### 1. Quản lý dữ liệu về công tác dân tộc

- **Tổ chức khai thác hiệu quả:**
  - Nâng cao hiệu quả quản lý nhà nước.
  - Số hóa quy trình, nghiệp vụ quản lý.

- **Thu thập và cập nhật dữ liệu:**
  - Ban Dân tộc tỉnh chủ trì, phối hợp với các cơ quan liên quan.
  - Thu thập, số hóa, cập nhật dữ liệu ban đầu; cập nhập khi có phát sinh, thay đổi.

- **Kết nối và đồng bộ dữ liệu:**
  - Kết nối CSDL về công tác dân tộc với Kho dữ liệu dùng chung của tỉnh.

- **Chia sẻ dữ liệu:**
  - Chia sẻ dữ liệu mặc định thông qua dịch vụ chia sẻ.
  - Chia sẻ theo yêu cầu đặc thù của cơ quan, đơn vị.

### 2. Quản lý dữ liệu về khu kinh tế và khu công nghiệp

- **Tổ chức khai thác hiệu quả:**
  - Nâng cao hiệu quả quản lý nhà nước.
  - Số hóa quy trình, nghiệp vụ quản lý.

- **Thu thập và cập nhật dữ liệu:**
  - Ban Quản lý Khu ki

In [ ]:
#### TODO:
#### https://github.com/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/retrieval_with_feedback_loop.ipynb
### Rerank with cross-encoder model https://github.com/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/reranking.ipynb
